# Coronagraph inpainting model

## Data downloading and preprocessing

In [1]:
!python3 data.py

/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
Traceback (most recent call last):
  File "/home/jenci/.local/lib/python3.10/site-packages/aiohttp/connector.py", line 992, in _wrap_create_connection
    return await self._loop.create_connection(*args, **kwargs)
  File "/usr/lib/python3.10/asyncio/base_events.py", line 1076, in create_connection
    raise exceptions[0]
  File "/usr/lib/python3.10/asyncio/base_events.py", line 1060, in create_connection
    sock = await self._connect_sock(
  File "/usr/lib/python3.10/asyncio/base_events.py", line 969, in _connect_sock
    await self.sock_connect(sock, address)
  File "/usr/lib/python3.10/asyncio/selector_events.py", line 501, in sock_connect
    return await fut
  File "/usr/lib/python3.10/asyncio/selector_events.py", line 541, in _sock_connect_cb
    raise OSError(err, f'Connect call failed {address}')
ConnectionRefusedError: [Errno 11

In [ ]:
from utils import get_default_device
device = get_default_device()
print(device)

## Creating dataset

In [ ]:
batch_size = 1

In [ ]:
from torch.utils.data import random_split, DataLoader
from utils import DeviceDataLoader
from data import CoronagraphDataset

dataset = CoronagraphDataset('c2')
# 0.8 - 0.2
train_len = round(0.8*len(dataset))
val_len = len(dataset) - train_len

#random split
train_ds, val_ds = random_split(dataset, [train_len, val_len])

train_dl = DeviceDataLoader(DataLoader(train_ds, batch_size, True, num_workers = 4, pin_memory=True), device)
val_dl = DeviceDataLoader(DataLoader(val_ds, batch_size*2, True, num_workers = 4, pin_memory=True), device)

In [ ]:
from model import CoronagraphReconstructor
from utils import to_device

model = to_device(CoronagraphReconstructor('c2', in_channels = 1, deep = 2, layers = 2), device)

In [ ]:
import torch
sample_tensor = (torch.randn(1,1,1024,1024), torch.ones(1,1,1024,1024))
sample_tensor = to_device(sample_tensor, device)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim import Adam

model.fit(
    train_dl,
    val_dl,
    epochs = 25,
    lr = 1e-5,
    batch_size = batch_size,
    weight_decay = 0,
    grad_clip = 1e-4,
    opt_func = Adam,
    lr_sched = OneCycleLR,
    saving_div = 5,
    graph = True,
    sample_input = sample_tensor
)